# Quantifying solidification of metallic alloys with scikit-image

<center><h2>C. Gus Becker</h2></center>
<center>
    <img src="images/csm_logo.png" width="400">
    <img src="images/canfsa_logo.png" width="300">
</center>
<center><h3>&</h3></center>
<center><h2>Marianne Corvellec</h2></center>
<center>
    <img src="images/skimage_logo.png" width="300">
    <img src="images/igdore_logo.png" width="300">
</center>

#### BIDS ImageXD Conference — May 18, 2021 ― Session 4: Materials Science